# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.51,87,97,13.96,TF,1727290881
1,1,tolanaro,-25.0319,46.9987,24.04,78,75,8.23,MG,1727290883
2,2,novodvinsk,64.4165,40.8122,11.86,71,95,4.57,RU,1727290884
3,3,nemuro,43.3236,145.5750,16.10,81,100,6.93,JP,1727290886
4,4,ola,59.5833,151.2833,4.10,59,0,1.52,RU,1727290887


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
tiles = gv.tile_sources.OSM
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles=tiles,  
    size='humidity',   
    color='blue', 
    hover_cols=['City', 'Country', 'humidity'],  
    title='City Humidity Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 18) & (city_data_df['Max Temp'] <= 26)]


# Drop any rows with null values
cleaned_cities_df = filtered_cities_df.dropna()

# Display sample data
cleaned_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,tolanaro,-25.0319,46.9987,24.04,78,75,8.23,MG,1727290883
5,5,hamilton,39.1834,-84.5333,25.14,70,100,1.34,US,1727290889
10,10,adamstown,-25.0660,-130.1015,20.22,82,100,6.87,PN,1727290897
12,12,lodja,-3.4833,23.4333,25.19,71,95,1.68,CD,1727290900
13,13,west island,-12.1568,96.8225,25.99,89,20,3.60,CC,1727290903
...,...,...,...,...,...,...,...,...,...,...
540,540,saveh,35.0213,50.3566,25.58,32,0,4.21,IR,1727291630
544,544,fort mcmurray,56.7268,-111.3810,19.77,55,100,2.06,CA,1727291635
547,547,antalaha,-14.9003,50.2788,20.63,83,0,3.47,MG,1727291639
548,548,chinde,26.8212,87.7095,23.04,93,100,1.86,NP,1727291640


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,tolanaro,MG,-25.0319,46.9987,78,
5,hamilton,US,39.1834,-84.5333,70,
10,adamstown,PN,-25.0660,-130.1015,82,
12,lodja,CD,-3.4833,23.4333,71,
13,west island,CC,-12.1568,96.8225,89,
...,...,...,...,...,...,...
540,saveh,IR,35.0213,50.3566,32,
544,fort mcmurray,CA,56.7268,-111.3810,55,
547,antalaha,MG,-14.9003,50.2788,83,
548,chinde,NP,26.8212,87.7095,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius= 10000
params = {
        'radius': radius,
        'categories': "accommodation.hotel",
        'apiKey': geoapify_key
    }
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
hamilton - nearest hotel: North Vista Manor
adamstown - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
praia da vitoria - nearest hotel: Salles
rayadrug - nearest hotel: No hotel found
deschutes river woods - nearest hotel: WorldMark Bend - Seventh Mountain Resort
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
oruro - nearest hotel: HOTEL VIRGEN DEL SOCAVON
san vicente de moravia - nearest hotel: Hotel Ave del Paraiso
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
bethel - nearest hotel: Hampton Inn Danbury
kwakoa - nearest hotel: Mhako Hostel & Restaurant
lagoa - nearest hotel: Casa da Madrinha
mpika - nearest hotel: Melody Lodge
constantia - nearest hotel: Hotel Dali
manokwari - nearest hotel: Hotel Arfak
mount isa - nearest hotel: Ibis Styles
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George


,City,Country,Lat,Lng,Humidity,Hotel Name
1,tolanaro,MG,-25.0319,46.9987,78,Hôtel Mahavokey
5,hamilton,US,39.1834,-84.5333,70,North Vista Manor
10,adamstown,PN,-25.0660,-130.1015,82,No hotel found
12,lodja,CD,-3.4833,23.4333,71,No hotel found
13,west island,CC,-12.1568,96.8225,89,Cocos Village Bungalows
...,...,...,...,...,...,...
540,saveh,IR,35.0213,50.3566,32,هتل صدرا
544,fort mcmurray,CA,56.7268,-111.3810,55,Nomad Hotel and Suites
547,antalaha,MG,-14.9003,50.2788,83,Hôtel Florida
548,chinde,NP,26.8212,87.7095,93,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True, 
    tiles='OSM', 
    size='humidity',
    hover_cols=['city', 'country', 'Hotel Name'],  
    title='Cities and Nearby Hotels',
    size_max=20,  
    color='blue' 
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)